In [66]:
import pandas as pd
pima = pd.read_csv('./01.Intro/pima-indians-diabetes.csv', skiprows=9, header=None)
pima.columns = ['P','G','BP','S','I','BMI','D','Age','Target']
pima.head()

,P,G,BP,S,I,BMI,D,Age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [67]:
X = pima.iloc[:, :-1].values
y = pima.Target.values

In [68]:
from sklearn.preprocessing import MinMaxScaler
X_std = MinMaxScaler().fit_transform(X)
X_std

array([[0.35294118, 0.74371859, 0.59016393, ..., 0.50074516, 0.23441503,
        0.48333333],
       [0.05882353, 0.42713568, 0.54098361, ..., 0.39642325, 0.11656704,
        0.16666667],
       [0.47058824, 0.91959799, 0.52459016, ..., 0.34724292, 0.25362938,
        0.18333333],
       ...,
       [0.29411765, 0.6080402 , 0.59016393, ..., 0.390462  , 0.07130658,
        0.15      ],
       [0.05882353, 0.63316583, 0.49180328, ..., 0.4485842 , 0.11571307,
        0.43333333],
       [0.05882353, 0.46733668, 0.57377049, ..., 0.45305514, 0.10119556,
        0.03333333]])

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_std, y, stratify=y, test_size=0.2, random_state=2023
)

In [70]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
svc = SVC(random_state=2023)
knn = KNeighborsClassifier()
lrc = LogisticRegression(random_state=2023)

In [74]:
svc = SVC(probability=True, random_state=2023)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.fit(X_train, y_train)

LogisticRegression(random_state=2023)

In [76]:
from sklearn.ensemble import VotingClassifier
voc = VotingClassifier(
    estimators=[('SVC',svc), ('KNN',knn), ('LRC',lrc)],
    voting='soft'
)

In [79]:
voc.fit(X_train, y_train)

VotingClassifier(estimators=[('SVC', SVC(probability=True, random_state=2023)),
                             ('KNN', KNeighborsClassifier()),
                             ('LRC', LogisticRegression(random_state=2023))],
                 voting='soft')

In [80]:
from sklearn.model_selection import GridSearchCV
params = {'SVC__C': [1.1, 1.3, 1.5], 'LRC__C': [0.5, 0.7, 0.9]}
grid_voc = GridSearchCV(voc, params, scoring='accuracy', cv=5)
grid_voc.fit(X_train, y_train)
grid_voc.best_params_

{'LRC__C': 0.7, 'SVC__C': 1.3}

In [88]:
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
accuracy_rfc = metrics.accuracy_score(y_test, pred_rfc)
precision_rfc = metrics.precision_score(y_test, pred_rfc)
recall_rfc = metrics.recall_score(y_test, pred_rfc)


In [89]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred_xgb= xgb.predict(X_test)
accuracy_xgb= metrics.accuracy_score(y_test, pred_xgb)
precision_xgb = metrics.precision_score(y_test, pred_xgb)
recall_xgb = metrics.recall_score(y_test, pred_xgb)

In [90]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
pred_lgb= lgb.predict(X_test)
accuracy_lgb= metrics.accuracy_score(y_test, pred_lgb)
precision_lgb = metrics.precision_score(y_test, pred_lgb)
recall_lgb = metrics.recall_score(y_test, pred_lgb)

In [93]:
print('\t\tRFC\tXGB\tLGB')
print(f'accuracy\t{accuracy_rfc:.4f}\t{accuracy_xgb:.4f}\t{accuracy_lgb:.4f}')
print(f'precision\t{precision_rfc:.4f}\t{precision_xgb:.4f}\t{precision_lgb:.4f}')
print(f'recall\t\t{recall_rfc:.4f}\t{recall_xgb:.4f}\t{recall_lgb:.4f}')

		RFC	XGB	LGB
accuracy	0.7597	0.7403	0.7273
precision	0.6735	0.6458	0.6200
recall		0.6111	0.5741	0.5741
